## __Text mining y Procesamiento de Lenguaje Natural (NLP)__

__Profesor__: Anthony D. Cho

__Tema__: Representación de documentos

__Método__: Modelo de espacio vectorial

****

__Dependencias__

```{python}
    python -m pip install nltk spacy
    python -m spacy download en_core_web_sm
    python -m spacy download es_core_news_sm
    
```

## Librerias

In [ ]:
from glob import glob
import re
from pandas import DataFrame
from numpy import zeros

from string import punctuation
from spacy.lang.es.stop_words import STOP_WORDS
from spacy import load

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Instancia del modelo de lenguaje
nlp = load('es_core_news_sm')

## Carga de documentos

In [ ]:
## Encontrar la ruta de cada archivo de interes
path_docs = glob('*/doc*.txt')

## Almacenamiendo de contenido de los documentos e id (nombre del archivo)
corpus, doc_id = [], [] 

## Incio de proceso de carga de documentos
if len(path_docs):
    for file in path_docs:

        ## Se carga el texto
        text = open(file, 'r', encoding='utf-8').read()
        
        ## Se almacena el texto
        corpus.append(text)
        
        id = file.split('\\')[-1].split('.')[0]

        ## Se almacena el id
        doc_id.append(id)
else:
    print('No corpus have found.')

#### Preprocesamiento

In [ ]:
## Limpieza de textos
cleanTexts = []

for doc in corpus:

    ## Remover numeros y puntuaciones
    doc = re.sub(r'[\"\¿\°\d+]', '', doc)
    doc = [s for s in doc if s not in punctuation]
    doc = ''.join(doc)

    ## Normalización y remover stopwords
    documento = nlp(doc.lower())
    tokens = [word.text for word in documento]
    doc = [word for word in tokens if word not in STOP_WORDS]
    doc = ' '.join(doc)
    doc = re.sub(pattern='\s+', repl=' ', string=doc)
    
    ## Aplicar lemmatización
    documento = nlp(doc)
    lemmas = [word.lemma_ for word in documento]
    doc = ' '.join(lemmas)
    doc = re.sub(pattern='\s+', repl=' ', string=doc)

    ## Almacenado de contenido procesado
    cleanTexts.append(doc)

## Mostar contenido procesado
cleanTexts
    

## Construcción de TF-IDF

In [ ]:
## Instancia del modelo
model = TfidfVectorizer(use_idf=True,      # <--
                        norm=None,          # <--
                        ngram_range=(1,1),
                        binary=False        
                        )

## Ajuste del modelo y retorno de TF matrix
tf_sparse = model.fit_transform(cleanTexts)

## Extraer Vocabulario creado por el modelo (dict :: key (word), value (index))
vocabulary = model.vocabulary_

In [ ]:
features = sorted(vocabulary.items(), key=lambda x: x[1])
features = [f for f, _ in features]

tf_table = DataFrame(tf_sparse.toarray(), columns=features, index=doc_id)
tf_table

#### Cosntrucción de un app prototipo para encontrar los 3 documentos más parecidos a una consulta realizada por un usuario

In [ ]:
consulta = input('Inserte lo que desea buscar').lower()
print(consulta)

In [ ]:
## Se codifica la consulta con el vocabulario generado por el modelo vectorial 
consulta_vsm = zeros(len(vocabulary))

## Se aplica lemmaticación
documento = nlp(consulta)
lemmas = [word.lemma_ for word in documento]

## Completar la inormación codificacda en el arreglo declarado "consulta_vsm"
for word in lemmas:
    try:
        vocabulary[word]
    except:
        print('word: {} not recognized.'.format(word))
    else:
        i = vocabulary[word]
        consulta_vsm[i] +=1

## Mostrar codificación
consulta_vsm = consulta_vsm*model.idf_
consulta_vsm


In [ ]:
## Se calcula similitud entre la consulta y los documentos
similitudes = cosine_similarity(X=consulta_vsm.reshape(1, -1), 
                                Y=tf_table)

similitudes = DataFrame(data={'similarity': similitudes.flatten(), 'docs': tf_table.index.to_list()})
similitudes = similitudes.sort_values(by='similarity', ascending=False)


In [ ]:
similitudes.head(3)

In [ ]:
## Mostrar el contenidos de los tres documentos encontrados
for i in range(3):

    doc = similitudes.iloc[i, 1]
    index = doc_id.index(doc)

    print(corpus[index])
    print('*'*60, end=3*'\n')